# Integral Equation Model Comparisons
Purpose of this notebook is to EOS calculations for comparisons to George Petrov's Average Atom model, and Morales EOS Hydrogen paper

[https://journals.aps.org/pre/abstract/10.1103/PhysRevE.81.021202](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.81.021202)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

from hnc.hnc.PIE import Plasma_of_Ions_and_Electrons as PLASMA

from hnc.hnc.constants import *
from hnc.hnc.misc import n_from_rs, rs_from_n

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
                
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}

</style>
""")


ModuleNotFoundError: No module named 'mendeleev'


#### Load George Data

In [2]:
GP_H_file = "H-rho=1.dat"
GP_df = read_csv(GP_H_file, delim_whitespace=True, header=0)
GP_PMbar_1 = GP_df['P_ishikawa'][3:]
GP_PMbar_2 = GP_df['P_fauss'][3:]
GP_Te_eV = GP_df['Te'][3:]

In [3]:
GP_Te_eV

3        1.0
4        2.0
5        5.0
6       10.0
7       20.0
8       50.0
9      100.0
10     200.0
11     500.0
12    1000.0
13    2000.0
Name: Te, dtype: float64

#### Load Morales Data

In [4]:
Morales_file = "Morales_EOS_data.dat"
Morales_df = read_csv(Morales_file, delim_whitespace=True, header=4)
Morales_EAU  = Morales_df['E[AU]']
Morales_PGPa = Morales_df['P[GPa]']
Morales_TK   = Morales_df['T[K]']
Morales_PMbar =  1e-2*Morales_PGPa
Morales_TeV = Morales_TK/eV_to_K

In [5]:
Morales_TeV

0    0.172347
1    0.258520
2    0.344693
3    0.517040
4    0.689386
5    0.861733
Name: T[K], dtype: float64

### Set Parameters in Paper and Plot

In [6]:
name = 'Hydrogen'
Z, A = 1,1 

Te_eV_list = np.geomspace(np.min(Morales_TeV), np.max(GP_Te_eV), num=15) #np.array(GP_Te_eV) # eV
Ti_eV_list = Te_eV_list
ni_cc = 5.98e23
ni_AU = ni_cc/cm_to_AU**3

ri_cm = rs_from_n(ni_cc)  # cm
ri_AU = ri_cm*cm_to_AU    # AU 

In [7]:
ri_AU

1.3914542638521432

In [8]:
from hnc.hnc.misc import Degeneracy_Parameter, find_η, P_Ideal_Fermi_Gas, Fermi_Energy

np.array([P_Ideal_Fermi_Gas(Te, ni_AU) for Te in Te_eV_list*eV_to_AU])*AU_to_bar/1e6

array([   9.92099106,    9.926066  ,    9.94538199,   10.01867366,
         10.29382884,   11.27418571,   14.24759619,   21.59603589,
         37.47601859,   69.72379761,  133.59938263,  258.93443737,
        504.00369643,  982.57296292, 1916.67816283])

In [11]:
np.sqrt((2/5 * Fermi_Energy(ni_AU)/(4/3*π*ri_AU**3) * AU_to_bar/1e6 )**2+( Te_eV_list*eV_to_AU*ni_AU* AU_to_bar/1e6)**2)

array([   9.92055719,    9.92441527,    9.93909189,    9.9947786 ,
         10.20403482,   10.96432281,   13.47209879,   20.36753499,
         36.10149733,   68.45711493,  132.54432755,  258.1030081 ,
        503.36959176,  982.0991122 , 1916.3265231 ])

#### Run Ornstein-Zernike Closure

In [10]:
def plasma_output(plasma):
    plasma.run_hnc()
    plasma.hnc.βu_r_matrix[1,1] -= plasma.βP_ee
    Pe_hnc_id, Pe_hnc_ex   = plasma.hnc.rho[1]*plasma.hnc.Temp_list[1],  np.sum( plasma.hnc.excess_pressure_matrix(),axis=1 )[1] 
    plasma.hnc.βu_r_matrix[1,1] += plasma.βP_ee
    Pe_Mbar_id, Pe_Mbar_ex = P_hnc_to_Mbar(Pe_hnc_id), P_hnc_to_Mbar(Pe_hnc_ex)
    Pe_Mbar = Pe_Mbar_id + Pe_Mbar_ex
    return Pe_Mbar_id, Pe_Mbar_ex, Pe_Mbar

name1 = 'OZ - g'
plasma1 = lambda T_eV: PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Z,
                 βu_options = {'add_bridge':False,'bridge':'ocp', 'pseudopotential':False}, 
                 qsp_options= {'r_c':0,'which_Tij':'geometric'},
                 hnc_options= {'kappa_multiscale':10, 'R_max':20,'N_bins':1000, 
                               'h_max':2e2,'oz_method':'standard'},
                 hnc_solve_options={'alpha_Picard':0.5, 'num_iterations':1e4})
name2 = 'OZ - th'
plasma2 = lambda T_eV: PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Z,
                 βu_options = {'add_bridge':False,'bridge':'ocp', 'pseudopotential':False}, 
                 qsp_options= {'r_c':0,'which_Tij':'thermal'},
                 hnc_options= {'kappa_multiscale':10, 'R_max':20,'N_bins':1000, 
                               'h_max':2e2,'oz_method':'standard'},
                 hnc_solve_options={'alpha_Picard':0.5, 'num_iterations':1e4})
name3 = 'OZ - th  Pauli'
plasma3 = lambda T_eV: PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Z, find_βuee=True,
                 βu_options = {'add_bridge':False,'bridge':'ocp', 'pseudopotential':False}, 
                 qsp_options= {'r_c':0,'which_Tij':'thermal'},
                 hnc_options= {'kappa_multiscale':10, 'R_max':20,'N_bins':1000, 
                               'h_max':2e2,'oz_method':'standard'},
                 hnc_solve_options={'alpha_Picard':0.5, 'num_iterations':1e4})
name4 = 'SVT - th'
plasma4 =  lambda T_eV: PLASMA(Z, A, ni_cc, T_eV, T_eV, Zbar=Z,
                 βu_options = {'add_bridge':False,'bridge':'ocp', 'pseudopotential':False}, 
                 qsp_options= {'r_c':0},
                 hnc_options= {'kappa_multiscale':10, 'R_max':20,'N_bins':1000, 
                               'h_max':2e2,'oz_method':'svt'},
                 hnc_solve_options={'alpha_Picard':0.5,'num_iterations':1e4})

P_hnc_to_bar = lambda P_hnc: (P_hnc/ri_AU**3) *  AU_to_bar 
P_hnc_to_Mbar = lambda P_hnc: 1e-6*P_hnc_to_bar(P_hnc)
P_hnc_to_GPa = lambda P_hnc:  P_hnc_to_bar(P_hnc) * 1e5 / 1e9 

####  Investigate Pressure 

In [11]:
# print("P = {0:.3e} [Mbar]".format(P_hnc_to_Mbar(Pid)))
# print("P = {0:.3e} [GPa]".format(P_hnc_to_GPa(Pid)))

### Compute all plasma models for list of densities

In [ ]:
P_list_1 = []
P_list_2 = []
P_list_3 = []
P_list_4 = []

for T_eV in Te_eV_list:
    #1
    # plasma = plasma1(T_eV)
    # Pe_id, Pe_ex, Pe = plasma_output(plasma)
    # P_list_1.append( [Pe_id, Pe_ex, Pe] )
    #2
    # plasma = plasma2(T_eV)
    # Pe_id, Pe_ex, Pe = plasma_output(plasma)
    # P_list_2.append( [Pe_id, Pe_ex, Pe] )
    #3
    plasma = plasma3(T_eV)
    Pe_id, Pe_ex, Pe = plasma_output(plasma)
    P_list_3.append( [Pe_id, Pe_ex, Pe] )
    #4
    # plasma = plasma4(T_eV)
    # Pe_id, Pe_ex, Pe = plasma_output(plasma)
    # P_list_4.append( [Pe_id, Pe_ex, Pe] )
    

Te_in_eV: 0.172
Ti_in_eV: 0.172


/home/zach/plasma/hnc/hnc/PIE.py:181: RuntimeWarning: overflow encountered in exp
  integrand = lambda t: t*np.sin(sin_arg*t)/(1+np.exp(t**2-η))
/home/zach/plasma/hnc/hnc/PIE.py:188: RuntimeWarning: overflow encountered in exp
  f_of_k = 1/(  1+np.exp((dense_hnc.k_array/self.qsp.ri)**2/(2*m_e*self.qsp.Te) - η) )


0: Initial condition Total Error: 1.064e+01
Exiting status 0, reverting to best index so far: 805
Final iter:806, Total Error: 9.922e-09
Te_in_eV: 0.336
Ti_in_eV: 0.336
0: Initial condition Total Error: 6.742e+00
Exiting status 0, reverting to best index so far: 453
Final iter:454, Total Error: 9.680e-09
Te_in_eV: 0.656
Ti_in_eV: 0.656
0: Initial condition Total Error: 4.187e+00
Exiting status 0, reverting to best index so far: 263
Final iter:264, Total Error: 9.381e-09
Te_in_eV: 1.281
Ti_in_eV: 1.281
0: Initial condition Total Error: 2.542e+00
Exiting status 0, reverting to best index so far: 158
Final iter:159, Total Error: 8.992e-09
Te_in_eV: 2.499
Ti_in_eV: 2.499
0: Initial condition Total Error: 1.505e+00
Exiting status 0, reverting to best index so far: 98
Final iter:99, Total Error: 8.765e-09
Te_in_eV: 4.876
Ti_in_eV: 4.876
0: Initial condition Total Error: 8.717e-01
Exiting status 0, reverting to best index so far: 63
Final iter:64, Total Error: 8.005e-09
Te_in_eV: 9.514
Ti_in_

/home/zach/plasma/hnc/hnc/PIE.py:182: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  κ = 3*(2*self.qsp.Te*m_e) / (self.qsp.k_F**3 * r) *quad(integrand, 0, t_max)[0]


0: Initial condition Total Error: 1.730e-01
Exiting status 0, reverting to best index so far: 25
Final iter:26, Total Error: 6.240e-09
Te_in_eV: 70.693
Ti_in_eV: 70.693


/home/zach/plasma/hnc/hnc/PIE.py:182: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  κ = 3*(2*self.qsp.Te*m_e) / (self.qsp.k_F**3 * r) *quad(integrand, 0, t_max)[0]


0: Initial condition Total Error: 9.698e-02
Exiting status 0, reverting to best index so far: 23
Final iter:24, Total Error: 9.712e-09
Te_in_eV: 137.946
Ti_in_eV: 137.946


Now Plot 

In [ ]:
# P_list_1 = np.array(P_list_1)
# P_list_2 = np.array(P_list_2)
P_list_3 = np.array(P_list_3)
# P_list_4 = np.array(P_list_4)

In [ ]:
fig, axs = plt.subplots( figsize=(10,8))
axs = np.array([axs])
# fig.suptitle(r"$T_i $ eV", fontsize=25, y=0.9)

#Plot GP data 
axs[0].plot(GP_Te_eV, GP_PMbar_1, 'k-', label='GP Ishikawa')
axs[0].plot(GP_Te_eV, GP_PMbar_2, 'k--', label='GP Fauss')

#Plot Morales data
axs[0].plot(Morales_TeV, Morales_PMbar, 'k-', label='Morales')

# Plot Pressure ratio
# axs[0].plot(GP_Te_eV, P_list_1[:,2], label=name1)
# axs[0].plot(Te_eV_list, P_list_2[:,2], label=name2)
axs[0].plot(Te_eV_list, P_list_3[:,2],'--', label=name3)
# axs[0].plot(Te_eV_list, P_list_4[:,2],'--.', label=name4)

# Plot Fermi Pressure
P_Fermi_Mbar = 2/5 * plasma.qsp.E_F/(4/3*π*(plasma.qsp.ri)**3) * AU_to_bar/1e6
axs[0].plot(GP_Te_eV, GP_Te_eV*0 +P_Fermi_Mbar,'r--', label=r"$P_{Fermi}$ [Mbar]"  )

for ax in axs:
    ax.legend(fontsize=15, loc='upper left')
    ax.set_xscale('log')
    ax.set_yscale('symlog', linthresh=1)
    ax.set_ylim(1e0, 1e5)
    ax.set_ylabel(r"$P_e$ [Mbar]", fontsize=20)
    ax.set_xlabel(r"$T$ [eV]", fontsize=20)
    ax.tick_params(labelsize=15)
plt.tight_layout()